In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
%cd "/content/gdrive/MyDrive/deep_learning_project/"
!pip install import-ipynb

/content/gdrive/MyDrive/deep_learning_project
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=84e6de07f0beebe6d675419b2f9936dec6e4b93d21d92bfeb2d7a6597238dc5b
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
import import_ipynb
from utils import *
import numpy as np
from tensorflow.keras import models

PATH_MODEL_1 = "/content/gdrive/MyDrive/deep_learning_project/models/2.2/Scratch_CNN.h5"
PATH_MODEL_2 = "/content/gdrive/MyDrive/deep_learning_project/models/3.2/InceptionV3.h5"
PATH_MODEL_3 = "/content/gdrive/MyDrive/deep_learning_project/models/3.2/VGG16.h5"

def prediction_traduction(prediction):
  if(prediction>0.5):
    return 1
  else:
    return 0

def load_models():
  model_1 = models.load_model(PATH_MODEL_1)
  model_2 = models.load_model(PATH_MODEL_2)
  model_3 = models.load_model(PATH_MODEL_3)
  return model_1,model_2,model_3

def average_voting(prediction_1,prediction_2,prediction_3):
  return prediction_traduction(0.33 * (prediction_1 + prediction_2 + prediction_3))

def contest_prediction(testing_images):
  first_testing_images = enhance_image(testing_images)
  first_testing_images = filter_dataset(first_testing_images)

  second_testing_images = enhance_contrast(testing_images)

  first_testing_images = np.expand_dims(first_testing_images, -1)
  second_testing_images = np.expand_dims(second_testing_images, -1)

  first_testing_images = first_testing_images/65536
  second_testing_images = second_testing_images/65536

  model_1,model_2,model_3 = load_models()

  average_prediction = []

  prediction_1 = model_1.predict(first_testing_images)
  prediction_2 = model_2.predict(second_testing_images)
  prediction_3 = model_3.predict(second_testing_images)

  for i in range(len(prediction_1)):
    if (average_voting(prediction_1[i],prediction_2[i],prediction_3[i])==0):
      average_prediction.append(1)
    else:
      average_prediction.append(0)
  return average_prediction

importing Jupyter notebook from utils.ipynb


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Class dictionary

- Original class labels have been assigned to the patches according to the following mapping:
  - 0: Baseline patch
  - 1: Mass, benign
  - 2: Mass, malignant
  - 3: Calcification, benign
  - 4: Calcification, malignant

- **For contest prediction, stick to the following mapping**:
  - 0: Benign
  - 1: Malignant

# How to submit a prediction



Adapt the following snippet to your program

In [ ]:
'''
test_images_private = np.load('your path here.private_test_tensor.npy'))

x_test_baseline = x_test_private[0::2] 
x_test_abno = x_test_private[1::2] # remove baseline if you don't use it

# [TODO] your normalization / preprocessing here

prediction = model.predict(x_test_abno) # include baseline if you use it
output_labels = prediction.argmax(axis = 1)

assert len(output_labels) == 336 # must be of this size!

with open('your path here.Submission.csv','w') as mf:
  mf.writelines("\n".join(str(i) for i in output_labels) )
'''

'\ntest_images_private = np.load(\'your path here.private_test_tensor.npy\'))\n\nx_test_baseline = x_test_private[0::2] \nx_test_abno = x_test_private[1::2] # remove baseline if you don\'t use it\n\n# [TODO] your normalization / preprocessing here\n\nprediction = model.predict(x_test_abno) # include baseline if you use it\noutput_labels = prediction.argmax(axis = 1)\n\nassert len(output_labels) == 336 # must be of this size!\n\nwith open(\'your path here.Submission.csv\',\'w\') as mf:\n  mf.writelines("\n".join(str(i) for i in output_labels) )\n'

In [ ]:
test_images_private = np.load('/content/gdrive/MyDrive/deep_learning_project/numpy data/private_test_tensor.npy')

x_test_baseline = test_images_private[0::2] 
x_test_abno = test_images_private[1::2] # remove baseline if you don't use it

prediction = contest_prediction(x_test_abno)
#prediction = model.predict(x_test_abno) # include baseline if you use it

In [ ]:
#output_labels = np.argmax(prediction,axis = 1)
output_labels = prediction

assert len(output_labels) == 336 # must be of this size!

with open('/content/gdrive/MyDrive/deep_learning_project/Submission.csv','w') as mf:
  mf.writelines("\n".join(str(i) for i in output_labels) )

# How do we evaluate a submission


Read the submission

In [ ]:
path = '/content/gdrive/MyDrive/deep_learning_project/'
y_pred = list(pd.read_csv(os.path.join(path,"Submission.csv"),header = None)[0].values)
assert len(y_pred) == 336 # must be of this size!

Read the true labels

In [ ]:
labels = np.load(os.path.join(path,'private_test_labels.npy'))
dict_map = {1:0,3:0,2:1,4:1}
y_true = np.asarray([dict_map[x] for x in labels[labels!=0]])

In [ ]:
accuracy_score(y_true,y_pred)